In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import sys
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_rec = pd.read_csv('recipes_10k.csv')
df_rec.head(12)

,Unnamed: 0,title,ingredients,directions,link,source,NER,site
0,1934604,Hot Spiked Cider,"[""8 large dried apple rings"", ""Small handful d...","[""Place the dried apples and currants in a lar...",www.cookstr.com/recipes/hot-spiked-cider,Recipes1M,"[""apple cider"", ""dark rum"", ""calvados"", ""nutme...",www.cookstr.com
1,1947433,Zucchini Spread,"[""2 teaspoons wine vinegar"", ""1 teaspoon sugar...","[""Put wine vinegar, sugar, and salt in a zip-t...",www.food.com/recipe/zucchini-spread-443492,Recipes1M,"[""wine vinegar"", ""sugar"", ""onion"", ""white pepp...",www.food.com
2,1976795,Fresh Pea Relish with Tuna Steak,"[""4 (6-ounce) tuna steaks"", ""Salt and pepper"",...","[""Preheat grill or broiler."", ""Season tuna wit...",www.foodnetwork.com/recipes/emeril-lagasse/fre...,Recipes1M,"[""peas"", ""fresh mixed baby greens"", ""extra-vir...",www.foodnetwork.com
3,400797,Hamburger Help Mate,"[""1 to 3 c. noodles"", ""1 can cream of chicken ...","[""Brown hamburger and onion."", ""Cook noodles.""...",www.cookbooks.com/Recipe-Details.aspx?id=853236,Gathered,"[""cream of chicken soup"", ""noodles"", ""garlic s...",www.cookbooks.com
4,1888762,Grape Jam,"[""1 1/2 bunches Grapes"", ""300 grams Granulated...","[""Wash the grapes one by one, cut in half, the...",cookpad.com/us/recipes/148776-grape-jam,Recipes1M,"[""grapes"", ""sugar"", ""lemon juice""]",cookpad.com
5,1221529,Pasta With Sardine Sauce (Pasta Con Sardine Si...,"[""1/2 lb sardine fillet"", ""1/4 cup extra virgi...","[""Cut fish into 1\"" pieces."", ""Saute garlic in...",www.food.com/recipe/pasta-with-sardine-sauce-p...,Gathered,"[""oregano"", ""tomato paste"", ""seedless raisin"",...",www.food.com
6,1546968,Mughlai Kheema Paratha,"[""3/4 cups Onion, Chopped Finely"", ""1/2 pounds...","[""Marinate the finely chopped onions with 1/2 ...",tastykitchen.com/recipes/sidedishes/mughlai-kh...,Gathered,"[""oil"", ""eggs"", ""\u00bc"", ""mutton"", ""cumin see...",tastykitchen.com
7,1139082,"Low Calorie Strawberry ""Pretzel"" Salad","[""Crust"", ""1 cup Fiber One cereal"", ""3 tablesp...","[""Combine crust ingredients and pat in the bot...",www.food.com/recipe/low-calorie-strawberry-pre...,Gathered,"[""cream cheese layer"", ""splenda sugar substitu...",www.food.com
8,1574434,Coffee And Cream Shortbreads,"[""2/3 cups Sugar"", ""1 cup Butter, Cold And Cut...","[""Set oven to 350\u00b0F."", ""Put sugar in the ...",tastykitchen.com/recipes/desserts/coffee-and-c...,Gathered,"[""sugar"", ""vanilla"", ""\u00bc"", ""all-purpose"", ...",tastykitchen.com
9,1047618,Raw Cheez Itz,"[""1 cup sunflower seeds, soaked in water 4 hou...","[""Requires preparation! Soak sunflower seeds, ...",www.food.com/recipe/raw-cheez-itz-347413,Gathered,"[""cumin"", ""tomatoes"", ""flax seed"", ""red pepper...",www.food.com


In [3]:
print(len(df_rec['title']))
df_rec = df_rec.drop_duplicates(subset="title")
print(len(df_rec['title']))

10000
9034


In [4]:
# this is a part where the ingredients of the recipes are extracted to get the information of type, unit, ingredient
import re

pattern_with_parenthese = r"^(\d+)\s*([\d\s\/]+)?\s*\((\d+\s*(?:\d+\/\d+)?)\s*(oz\.?)\.\)\s*([^\r\n]+)$"
pattern_without_parenthese = r"^(\d+\s\d+/\d+ |\d+\s\w+\s\d+ |\d+ |\d+/\d+)\s*(\S+)?\s*([^\r\n]+)$"

count = 0
ingredient_count = 0
recipe_dic = {}

#go to whole dataset and look at exceptions of the regrex expression
def reportExceptions(matches2, ingredient):
  global count
  if (matches2):
    return
  else:
    count = count + 1
    print(ingredient)

def normailzeUnits(unit:str):
  unit = unit.replace('ounces', 'oz.')
  unit = unit.replace('c.', 'cups')
  
  if (len(unit) > 0 and not unit[0].isdigit()):
    unit = "1 " + unit
  return unit


def extractUnits(row):
  ingredients = row['ingredients']
  recipe_title = row['title']
  ingredients = ingredients.replace('"', '').replace('[', '').replace(']', '')
  ingredients = ingredients.split(',')
  global ingredient_count
  global count
  
  for i in ingredients:
    ingredient_count = ingredient_count + 1
    n = i.lstrip()
    n = normailzeUnits(n)
    # print(i)
    matches1 = re.match(pattern_with_parenthese, n)
    matches2 = re.match(pattern_without_parenthese, n)
    
    if (not matches1 and not matches2):
      continue
    elif (matches1):
      amount = matches1.group(1) if matches1.group(1) else 0
      unit = matches1.group(3) if matches1.group(3) else "NA"
      ingredient = matches1.group(4) if matches1.group(4) else "NA"
      if(recipe_title in recipe_dic):
        recipe_dic[recipe_title].append({"amount": amount, "unit": unit, "ingredient": ingredient})
      else:
        recipe_dic[recipe_title] = [{"amount": amount, "unit": unit, "ingredient": ingredient}]
    elif (matches2):
      amount = matches2.group(1) if matches2.group(1) else 0
      unit = matches2.group(2) if matches2.group(2) else "NA"
      ingredient = matches2.group(3) if matches2.group(3) else "NA"
      if(recipe_title in recipe_dic):
        recipe_dic[recipe_title].append({"amount": amount, "unit": unit, "ingredient": ingredient})
      else:
        recipe_dic[recipe_title] = [{"amount": amount, "unit": unit, "ingredient": ingredient}]
      
      
# extractUnits(df_rec['ingredients'].iloc[3])
df_rec.apply(extractUnits, axis=1)
print("ingredient_count: ", ingredient_count)
print("count: ", count)

print(len(recipe_dic.items()))

ingredient_count:  97511
count:  0
9034


In [5]:
row = df_rec[df_rec['title'] == "Hot Spiked Cider"]
list(row['ingredients'])

['["8 large dried apple rings", "Small handful dried currants (optional)", "4 ounces calvados or applejack (see Straight Up)", "64 ounces fresh apple cider", "12 whole cloves", "6 whole star anise", "3 cinnamon sticks", "1/2 teaspoon freshly grated nutmeg", "2 tablespoons brown sugar for dredging the apple slices", "24 ounces dark rum", "8 extra-long cinnamon sticks"]']

In [6]:
recipe_dic["Hot Spiked Cider"]

[{'amount': '8 ', 'unit': 'large', 'ingredient': 'dried apple rings'},
 {'amount': '1 ',
  'unit': 'Small',
  'ingredient': 'handful dried currants (optional)'},
 {'amount': '4 ',
  'unit': 'oz.',
  'ingredient': 'calvados or applejack (see Straight Up)'},
 {'amount': '64 ', 'unit': 'oz.', 'ingredient': 'fresh apple cider'},
 {'amount': '12 ', 'unit': 'whole', 'ingredient': 'cloves'},
 {'amount': '6 ', 'unit': 'whole', 'ingredient': 'star anise'},
 {'amount': '3 ', 'unit': 'cinnamon', 'ingredient': 'sticks'},
 {'amount': '1/2', 'unit': 'teaspoon', 'ingredient': 'freshly grated nutmeg'},
 {'amount': '2 ',
  'unit': 'tablespoons',
  'ingredient': 'brown sugar for dredging the apple slices'},
 {'amount': '24 ', 'unit': 'oz.', 'ingredient': 'dark rum'},
 {'amount': '8 ', 'unit': 'extra-long', 'ingredient': 'cinnamon sticks'}]

In [15]:
with open ("recipe_ingredients", "w") as f:
  for k, value in recipe_dic.items():
    for i in value:
      f.write(i["ingredient"]+"\n")
    
    
    with open ("nutrition_description", "w") as f:
  f.write("\n".join (list(data['description'].astype(str))))

dried apple rings
handful dried currants (optional)
calvados or applejack (see Straight Up)
fresh apple cider
cloves
star anise
sticks
freshly grated nutmeg
brown sugar for dredging the apple slices
dark rum
cinnamon sticks
wine vinegar
sugar
salt
i
(medium size)
(small finely)
parsley sprig
h
chives
h
cream cheese
salt (extra)
pepper
red capsicum
& coarsely chopped
tuna steaks
and pepper
oil
grilling
fresh shelled peas
chopped shallots
red pepper
chopped fresh mint
extra-virgin olive oil
n
e
d
fresh mixed baby greens
and thoroughly dried
parsley
garnish
noodles
cream of chicken soup
vegetarian vegie soup
oz
hamburger
n
pepper
salt (if you want)
Grapes
Granulated sugar
Lemon juice
sardine fillet
extra virgin olive oil
cloves
sliced
ounce) can tomato paste
seedless raisin (optional)
fresh oregano (dried herbs may be used)
basil leaves (dried herbs may be used)
cheese (fresh is best) or romano cheese
(fresh is best)
and pepper
pasta
Onion
Finely
s
.
Mutton Mince
Turmeric
d
Red Chili Powd

In [7]:
# print(df_rec["title"].head(12))
for t in df_rec["title"]:
  for u in recipe_dic[t]:
    print(str(u['unit']))


large
Small
oz.
oz.
whole
whole
cinnamon
teaspoon
tablespoons
oz.
extra-long
teaspoons
teaspoon
teaspoon
zucchin
grated
onion
bunch
fres
bunch
fres
g
teaspoon
white
cup
roasted
(6-ounce)
Salt
Olive
for
cup
tablespoons
tablespoon
teaspoons
teaspoons
tablespoo
orang
halve
cups
washed
Chopped
for
cups
can
can
8 
lb.
onio
green
garlic
bunches
grams
tsp
lb
cup
garlic
thinly
(6
cup
sprig
fresh
parmesan
grated
salt
lb
cups
Chopped
pound
oz
weight
teaspoon
Divide
teaspoon
Divide
teaspoons
Divide
cup
cups
Tablespoons
Divide
cups
cups
whole
teaspoons
whole
whole
whole
whole
teaspoon
teaspoon
teaspoon
Tablespoon
Tablespoons
whole
whole
Chopped
Crus
cup
tablespoons
cup
sal
to
Cream
oz.
softene
cup
cups
Fruity
(6
cups
cups
rinsed
cups
cup
Cold
teaspoons
Tablespoon
teaspoon
Tablespoons
oz
weight
teaspoon
cup
soaked
cup
soaked
cup
soaked
tomatoe
cup
dice
cup
groun
pinch
teaspoons
cups
teaspoon
teaspoon
cups
can
Topping
cups
cup
cup
cup
Salt
4 
Monterey
egg
cups
using
cups
grate
sals
oz.
peeled
cups
c

tsp
tsp
buttery
stick
tsp.
tsp.
cups
cups
cups
cups
cups
cup
melte
cup
cup
eg
teaspoon
teaspoon
teaspoon
teaspoon
large
peeled
cup
cups
Tbsp.
Tbsp.
dash
dash
sticks
boxes
oz.
cooke
oz.
onio
choppe
bag
tablespoon
crispy
cup
cup
cup
cup
cups
tsp.
cups
cups
cups
cups
boxes
small
undraine
banana
boxes
large
4 
oz.
tablespoons
oz.
quartere
pieces
12
lb.
cups
grated
lemon
cups
quarts.
cups
pound
tbsp.
pt.
1/2
cups
cups
stick
melte
oz.
can
cups
3 
cups
10 
cup
large
thinly
cut
see
cup
tsp
tsp
chili
garlic
coriande
sal
oz
may
stick
12 
16 
box
lb
cup
flour
12
Shredde
lb.
peele
cut
slices
choppe
tomatoe
choppe
green
thinly
egg
tsp.
cup
cup
melte
plus
cups
teaspoon
teaspoon
large
large
cup
teaspoon
tablespoons
tablespoons
Confectioners
for
cup
envelope
brisket
tbsp
onio
choppe
cloves
slice
chopped
medium
sliced
tbsp
tsp
tsp
tsp
sal
peppe
cups
pkg.
crushed
cups
cups
cups
cups
cups
tablespoons
tablespoons
teaspoon
teaspoon
cup
cup
cup
cup
cup
cup
tablespoons
melte
bacon
cooked
tablespoon
large
yel

In [8]:
# row = df_rec[df_rec['title'] == "C. P. R.(Chili Pancake Rellenos)  "]
# list(row['ingredients'])

In [9]:
# row = df_rec[df_rec['title'] == df_rec["title"].iloc[7]]
# list(row['ingredients'])

In [10]:
# df_rec = pd.read_csv("archive/recipes_data.csv")
# df_rec = df_rec.sample(n=10000)
# df_rec.to_csv("recipes_10k.csv")

In [11]:
# read the file of nutrition data
Nutrition_file = pd.read_csv("Nutrition.csv")
Nutrition_file["description"] = Nutrition_file["description"].astype(str)


In [12]:
def matching_ingredients(ingredients: str, nutritions):

    ingredients = "".join([a for a in ingredients if a.isalpha() or a == " "])
    ingredients = ingredients.split()
    
    for x in ingredients:
        print(nutritions[nutritions["description"].str.contains(x)])
        print(x)

matching_ingredients("handful dried currants (optional)", Nutrition_file)

Empty DataFrame
Columns: [Unnamed: 0, description, food_category_id, nutrient_id, amount, data_points, derivation_id, min, max, median, id_y, name, unit_name, nutrient_nbr]
Index: []
handful
       Unnamed: 0                                        description  \
80             80                                   egg whites dried   
81             81                                    egg white dried   
274           274  mission figs dried region  na yes proximates   nf   
275           275  mission figs dried region  na yes proximates   nf   
276           276  figs dried pass  region  na no proximates  nfycdb   
...           ...                                                ...   
57648       57648  bartlett pears pass  freeze dried regions    n...   
57675       57675  mission figs dried region  na yes amino acids   n   
57676       57676  mission figs dried region  na yes amino acids   n   
57677       57677  figs dried pass  region  na no amino acids  nf...   
57678       57678

In [13]:
text = list(Nutrition_file["description"])
print(len(text))
text

62659


['hummus',
 'milk  ',
 'proximates beef eye of round roaststeak lean only raw animal  select',
 'proximates beef eye of round roaststeak lean only raw animal  select',
 'proximates beef eye of round roaststeak lean only raw animal  select',
 'proximates beef eye of round roaststeak lean only raw animal  select',
 'proximates beef eye of round roaststeak lean only raw animal  select',
 'proximates beef eye of round roaststeak lean only raw animal  select',
 'proximates beef eye of round roaststeak lean only raw animal  select',
 'proximates beef eye of round roaststeak lean only raw animal  select',
 'proximates beef eye of round roaststeak lean only raw animal  select',
 'proximates beef eye of round roaststeak lean only raw animal  select',
 'proximates beef eye of round roaststeak lean only raw animal  select',
 'proximates beef eye of round roaststeak lean only raw animal  select',
 'hummus commercial',
 'milk reduced fat fluid  milkfat with added vitamin a and vitamin d',
 'tomatoe